# Alias Generator Functions

When working with REST APIs, using camel case for deserializing/serializing data, and snake case for field names is extremely common.

One approach is to use the alias definitions we looked at in the previous video.

However, doing this for hundreds of fields and dozens of models is tedious and error prone.

Converting from snake case to camel case is rather systematic, and we could write a function to do this.

In fact, there are 3rd party libraries that provide this very capability.

Pydantic also does.

These case conversion functions are documented [here](https://docs.pydantic.dev/latest/api/config/#pydantic.alias_generators)

Let's look at them:

In [1]:
from pydantic.alias_generators import to_camel, to_snake, to_pascal

These are just regular functions:

In [2]:
to_camel("last_name")

'lastName'

In [3]:
to_snake("lastName")

'last_name'

In [4]:
to_pascal("last_name")

'LastName'

The reason we have those functions is that we can configure our **model** to auto generate field aliases using one of those functions.

But any function that converts one string (the field name) into another (an alias) can also be used.

Let's take a look at auto-generating aliases that simply uppercases the field names.

In [5]:
def make_upper(in_str: str) -> str:
    return in_str.upper()

Just a regular function:

In [6]:
make_upper("last_name")

'LAST_NAME'

Now let's attach this function to our model definition, using, of course, `model_config`:

In [7]:
from pydantic import BaseModel, ConfigDict, Field, ValidationError

class Person(BaseModel):
    model_config = ConfigDict(alias_generator=make_upper)

    id_: int
    first_name: str | None = None
    last_name: str
    age: int | None = None

Let's inspect the model fields:

In [8]:
Person.model_fields

{'id_': FieldInfo(annotation=int, required=True, alias='ID_'),
 'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, alias='FIRST_NAME'),
 'last_name': FieldInfo(annotation=str, required=True, alias='LAST_NAME'),
 'age': FieldInfo(annotation=Union[int, NoneType], required=False, alias='AGE')}

You can see that the aliases were auto generated. Let's try it out:

In [9]:
p = Person(ID_=1, LAST_NAME="Fourier", AGE=62)
p

Person(id_=1, first_name=None, last_name='Fourier', age=62)

In [10]:
p.model_dump()

{'id_': 1, 'first_name': None, 'last_name': 'Fourier', 'age': 62}

In [11]:
p.model_dump(by_alias=True)

{'ID_': 1, 'FIRST_NAME': None, 'LAST_NAME': 'Fourier', 'AGE': 62}

You'll notice how `id_`'s alias became `ID_` - maybe we don't want that, in which case we can override it:

In [12]:
class Person(BaseModel):
    model_config = ConfigDict(alias_generator=make_upper)

    id_: int = Field(alias="ID")
    first_name: str | None = None
    last_name: str
    age: int | None = None

In [13]:
Person.model_fields

{'id_': FieldInfo(annotation=int, required=True, alias='ID', alias_priority=2),
 'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, alias='FIRST_NAME'),
 'last_name': FieldInfo(annotation=str, required=True, alias='LAST_NAME'),
 'age': FieldInfo(annotation=Union[int, NoneType], required=False, alias='AGE')}

In [14]:
p = Person(ID=1, LAST_NAME="Fourier", AGE=62)
p

Person(id_=1, first_name=None, last_name='Fourier', age=62)

In [15]:
p.model_dump()

{'id_': 1, 'first_name': None, 'last_name': 'Fourier', 'age': 62}

In [16]:
p.model_dump(by_alias=True)

{'ID': 1, 'FIRST_NAME': None, 'LAST_NAME': 'Fourier', 'AGE': 62}

Normally, we simply want our aliases to be camel case versions of the field names - so we can simply use on of Pydantic's utility functions, the `to_camel` function:

In [17]:
class Person(BaseModel):
    model_config = ConfigDict(alias_generator=to_camel)

    id_: int = Field(alias="id")
    first_name: str | None = None
    last_name: str
    age: int | None = None

We can inspect the fields:

In [18]:
Person.model_fields

{'id_': FieldInfo(annotation=int, required=True, alias='id', alias_priority=2),
 'first_name': FieldInfo(annotation=Union[str, NoneType], required=False, alias='firstName'),
 'last_name': FieldInfo(annotation=str, required=True, alias='lastName'),
 'age': FieldInfo(annotation=Union[int, NoneType], required=False, alias='age')}

As you can see we have the desired aliases, and use them in the usual manner:

In [19]:
p = Person(id=1, lastName="Fourier", age=62)
p

Person(id_=1, first_name=None, last_name='Fourier', age=62)

In [20]:
p.model_dump()

{'id_': 1, 'first_name': None, 'last_name': 'Fourier', 'age': 62}

In [21]:
p.model_dump(by_alias=True)

{'id': 1, 'firstName': None, 'lastName': 'Fourier', 'age': 62}

It is customary, when the data we are deserializing contains Python reserved words, to simply define the field name as that name with an underscore (`_`) appended to the field name, e.g.:
- `id` -> `id_`
- `list` -> `list_`
- `filter` -> `filter_`

We could certainly use the method we just saw, setting an alias generator, and overriding these special cases one by one:

In [22]:
class Model(BaseModel):
    model_config = ConfigDict(alias_generator=to_camel)

    id_: int = Field(alias="id")
    list_: list[str] = Field(alias="list")
    filter_: dict = Field(alias="filter")
    number_elements: list[int]

When we do this we have the following aliases:

In [23]:
Model.model_fields

{'id_': FieldInfo(annotation=int, required=True, alias='id', alias_priority=2),
 'list_': FieldInfo(annotation=list[str], required=True, alias='list', alias_priority=2),
 'filter_': FieldInfo(annotation=dict, required=True, alias='filter', alias_priority=2),
 'number_elements': FieldInfo(annotation=list[int], required=True, alias='numberElements')}

But since we are completely consistent with how we generate these aliases (basically we convert to camel case, then strip our any trailing underscore), we could actually do this completely with a custom alias generator function:

In [24]:
def make_alias(field_name: str) -> str:
    alias = to_camel(field_name)
    return alias.removesuffix("_")

In [25]:
make_alias("id_")

'id'

In [26]:
make_alias("number_elements")

'numberElements'

And let's use this unction for our model now:

In [27]:
class Model(BaseModel):
    model_config = ConfigDict(alias_generator=make_alias)

    id_: int
    list_: list[str]
    filter_: dict
    number_elements: list[int]

In [28]:
Model.model_fields

{'id_': FieldInfo(annotation=int, required=True, alias='id'),
 'list_': FieldInfo(annotation=list[str], required=True, alias='list'),
 'filter_': FieldInfo(annotation=dict, required=True, alias='filter'),
 'number_elements': FieldInfo(annotation=list[int], required=True, alias='numberElements')}